In [206]:
from collections import OrderedDict

class MatchingEngine():
    def __init__(self):
        self.bid_book = [] 
        self.ask_book = [] 
        
        self.bid_book_dict = OrderedDict()
        self.ask_book_dict = OrderedDict()

    def handle_order(self, order):
        print('MATCHING ENGINE IS WORKING')
        if order.type == OrderType.LIMIT:
            MatchingEngine.handle_limit_order(self, order)
        elif order.type == OrderType.MARKET:
            MatchingEngine.handle_market_order(self, order)
        elif order.type == OrderType.IOC:
            MatchingEngine.handle_ioc_order(self, order)
        else:
            raise UndefinedOrderType("Undefined Order Type!")

    def handle_limit_order(self, order): 
        filled_orders = []
        if order.side == OrderSide.BUY: # buy at a specified price or price which is lower than specified

            best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None

            while best_price is not None and (order.price == 0 or order.price >= best_price) \
                    and order.quantity >= 1e-9:
                best_price_qty = sum([ask.quantity for ask in self.ask_book_dict[best_price]]) # !!!!!!!!
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty > 0, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.ask_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(LimitOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))
                    
                    filled_orders.append(LimitOrder(order.id, order.symbol, match_qty, order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.ask_book_dict[best_price][0]
                
                if len(self.ask_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.ask_book_dict[best_price]
                best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None
            
            if order.quantity > 0:
                self.insert_limit_order(order)

        elif order.side == OrderSide.SELL: # sell at a specified price of price which is higher than specified
            best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
            
            while best_price is not None and (order.price == 0 or order.price <= best_price) and \
                  order.quantity >= 1e-9:
                best_price_qty = sum([bid.quantity for bid in self.bid_book_dict[best_price]])
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty >= 1e-9, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.bid_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(LimitOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))
                    
                    filled_orders.append(LimitOrder(order.id, order.symbol, match_qty, order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.bid_book_dict[best_price][0]
                            
                if len(self.bid_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.bid_book_dict[best_price]
                best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
                
            if order.quantity > 0:
                self.insert_limit_order(order)
                
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
        
        self.bid_book = []
        for key, val in enumerate(self.bid_book_dict):
            self.bid_book.append(self.bid_book_dict[val][0])
        self.bid_book = self.bid_book[::-1]
            
        self.ask_book = []   
        for key, val in enumerate(self.ask_book_dict):
            self.ask_book.append(self.ask_book_dict[val][0])
        self.ask_book = self.ask_book[::-1]
        
        return filled_orders

    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        
        if order.side == OrderSide.SELL:
            depth = self.ask_book_dict.setdefault(order.price, [])
            depth.append(order)
            
            self.ask_book = []   
            for key, val in enumerate(self.ask_book_dict):
                self.ask_book.append(self.ask_book_dict[val][0])
            
        elif order.side == OrderSide.BUY:
            depth = self.bid_book_dict.setdefault(order.price, [])
            depth.append(order)
            self.ask_book = self.ask_book[::-1]
            
            self.bid_book = []
            for key, val in enumerate(self.bid_book_dict):
                self.bid_book.append(self.bid_book_dict[val][0])
            self.bid_book = self.bid_book[::-1]

        else:
            raise UndefinedOrderSide("Undefined Order Side!") 
            
            
    def handle_market_order(self, order): 
        filled_orders = []
        if order.side == OrderSide.BUY: # buy at the best possible price

            best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None

            while best_price is not None and order.quantity >= 1e-9:
                best_price_qty = sum([ask.quantity for ask in self.ask_book_dict[best_price]]) # !!!!!!!!
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty > 0, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.ask_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(LimitOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))

                    filled_orders.append(LimitOrder(order.id, order.symbol, match_qty, match_order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.ask_book_dict[best_price][0]
                            
                if len(self.ask_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.ask_book_dict[best_price]
                best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None

        elif order.side == OrderSide.SELL: # sell at a specified price of price which is higher than specified
            best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
            
            while best_price is not None and order.quantity >= 1e-9:
                best_price_qty = sum([bid.quantity for bid in self.bid_book_dict[best_price]])
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty >= 1e-9, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.bid_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(LimitOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))
                    
                    filled_orders.append(LimitOrder(order.id, order.symbol, match_qty, match_order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.bid_book_dict[best_price][0]
                
                if len(self.bid_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.bid_book_dict[best_price]
                best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
                
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
        
        self.bid_book = []
        for key, val in enumerate(self.bid_book_dict):
            self.bid_book.append(self.bid_book_dict[val][0])
        self.bid_book = self.bid_book[::-1]
            
        self.ask_book = []   
        for key, val in enumerate(self.ask_book_dict):
            self.ask_book.append(self.ask_book_dict[val][0])
        self.ask_book = self.ask_book[::-1]
        
        return filled_orders
    
    def handle_ioc_order(self, order): 
        filled_orders = []
        if order.side == OrderSide.BUY: # buy at a specified price or price which is lower than specified

            best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None

            while best_price is not None and (order.price == 0 or order.price >= best_price) \
                    and order.quantity >= 1e-9:
                best_price_qty = sum([ask.quantity for ask in self.ask_book_dict[best_price]]) # !!!!!!!!
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty > 0, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.ask_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(IOCOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))
                    
                    filled_orders.append(IOCOrder(order.id, order.symbol, match_qty, order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.ask_book_dict[best_price][0]
                            
                
                if len(self.ask_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.ask_book_dict[best_price]
                best_price = min(self.ask_book_dict.keys()) if len(self.ask_book_dict) > 0 else None
            
            if order.quantity > 0:
                self.insert_limit_order(order)

        elif order.side == OrderSide.SELL: # sell at a specified price of price which is higher than specified
            best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
            
            while best_price is not None and (order.price == 0 or order.price <= best_price) and \
                  order.quantity >= 1e-9:
                best_price_qty = sum([bid.quantity for bid in self.bid_book_dict[best_price]])
                match_qty = min(best_price_qty, order.quantity)
                assert match_qty >= 1e-9, "Match quantity must be larger than zero"
                
                while match_qty >= 1e-9:
                    match_order = self.bid_book_dict[best_price][0] # !!!!!!!!
                    match_order_qty = min(match_qty, match_order.quantity) # !!!!!!!!
                    filled_orders.append(IOCOrder(match_order.id, match_order.symbol, match_order_qty, \
                                               match_order.price, match_order.side, match_order.time))
                    
                    filled_orders.append(IOCOrder(order.id, order.symbol, match_qty, order.price, \
                                                order.side, order.time))
                    order.quantity -= match_qty
                
                    match_order.quantity -= match_order_qty
                    match_qty -= match_order_qty
                    if match_order.quantity < 1e-9:
                            del self.bid_book_dict[best_price][0]
                            
                            
                if len(self.bid_book_dict[best_price]) == 0: # if the price does not have orders, delete the particular price depth
                        del self.bid_book_dict[best_price]
                best_price = max(self.bid_book_dict.keys()) if len(self.bid_book_dict) > 0 else None
                
        else:
            raise UndefinedOrderSide("Undefined Order Side!")
        
        self.bid_book = []
        for key, val in enumerate(self.bid_book_dict):
            self.bid_book.append(self.bid_book_dict[val][0])
        self.bid_book = self.bid_book[::-1]
            
        self.ask_book = []   
        for key, val in enumerate(self.ask_book_dict):
            self.ask_book.append(self.ask_book_dict[val][0])
        self.ask_book = self.ask_book[::-1]
        
        return filled_orders
    
    def amend_quantity(self, id, quantity):
        for key, val in enumerate(self.bid_book_dict):
            if self.bid_book_dict[val][0].id == id:
                if self.bid_book_dict[val][0].quantity > quantity:
                    self.bid_book_dict[val][0].quantity = quantity
                else:
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                    
        for key, val in enumerate(self.ask_book_dict):
            if self.ask_book_dict[val][0].id == id:
                if self.ask_book_dict[val][0].quantity > quantity:
                    self.ask_book_dict[val][0].quantity = quantity
                else:
                    raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                    
        self.bid_book = []
        for key, val in enumerate(self.bid_book_dict):
            self.bid_book.append(self.bid_book_dict[val][0])
        self.bid_book = self.bid_book[::-1]
            
        self.ask_book = []   
        for key, val in enumerate(self.ask_book_dict):
            self.ask_book.append(self.ask_book_dict[val][0])
        self.ask_book = self.ask_book[::-1]

    def cancel_order(self, id):
        for key, val in enumerate(self.bid_book_dict):
            
            if self.bid_book_dict[val][0].id == id:
                del self.bid_book_dict[val]
                break
                
        for key, val in enumerate(self.ask_book_dict):
            if self.ask_book_dict[val][0].id == id:
                del self.ask_book_dict[val]
                break
                
        self.bid_book = []
        for key, val in enumerate(self.bid_book_dict):
            self.bid_book.append(self.bid_book_dict[val][0])
        self.bid_book = self.bid_book[::-1]
            
        self.ask_book = []   
        for key, val in enumerate(self.ask_book_dict):
            self.ask_book.append(self.ask_book_dict[val][0])
        self.ask_book = self.ask_book[::-1]

In [253]:
from collections import deque
import time
import random
from abc import ABC
from enum import Enum

class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2

class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass

class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC


class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit=False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit

        
trader_to_exchange = deque()
exchange_to_trader = [deque() for _ in range(100)] # 100 threads representing traders

# Above you are given two deques where the orders submitted 
# to the exchange and back to the trader
# are expected to be populated by the trading exchange simulator
# The first is trader_to_exchange, a deque of orders to be populated
# for the exchange to execute
# The second is a list of 100 deques exchange_to_trader, which are 
# acknowledgements from the exchange
# to each of the 100 traders for trades executed on their behalf

# Below you have an implementation of a simulated thread to be 
# used where each trader is a separate thread
class MyThread:
    list_of_threads=[]
    def __init__(self,id='NoID'):
        MyThread.list_of_threads.append(self)
        self.is_started=False
        self.id = id
    def start(self):
        self.is_started = True
    def join(self):
        print('Trader ' + str(self.id) + ' will be waited')
        
# Paste in your implementation for the matching engine below

# ----------------------------------------------------------
# PASTE MATCHING ENGINE FROM Q2 HERE
#-----------------------------------------------------------

# Each trader can take a separate action chosen from the list below:

# Actions:
# 1 - Place New Order/Order Filled
# 2 - Amend Quantity Of An Existing Order
# 3 - Cancel An Existing Order
# 4 - Return Balance And Position

# request - (Action #, Trader ID, Additional Arguments)

# result - (Action #, Action Return)

# WE ASSUME 'AAPL' IS THE ONLY TRADED STOCK.

class ActionType(Enum):
    PLACE = 1
    AMEND = 2
    CANCEL = 3
    BALANCE = 4
    
import time
class Trader(MyThread):
    def __init__(self, id):
        super().__init__(id)
        self.book_position = 0
        self.balance_track = [1000000]
        self.exchange = Exchange()
        # the traders each start with a balance of 1,000,000 and nothing on the books
        # each trader is a thread

    def place_limit_order(self, quantity=None, price=None, side=None):
        # Make sure you modify the book position after the trade
        self.book_position += 1
#         print('trader {} book position'.format(self.id), self.book_position)
        order = LimitOrder(self.id, "AAPL", quantity, price, side, time.time())
        send = (ActionType.PLACE, self.id, order)
        print('trader sent:', send)
        return send

    def place_market_order(self, quantity=None, side=None):
        self.book_position += 1
#         print('trader {} book position'.format(self.id), self.book_position)
        order = MarketOrder(self.id, "AAPL", quantity, side, time.time())
        send = (ActionType.PLACE, self.id, order)
        print('trader sent:', send)
        return send

    def place_ioc_order(self, quantity=None, price=None, side=None):
        self.book_position += 1
#         print('trader {} book position'.format(self.id), self.book_position)
        order = IOCOrder(self.id, "AAPL", quantity, price, side, time.time())
        send = (ActionType.PLACE, self.id, order)
        print('trader sent:', send)
        return send
        
    def amend_quantity(self, quantity=None):
        send = (ActionType.AMEND, self.id, quantity)
        print('trader sent:', send)
        return send

    def cancel_order(self):
        send = (ActionType.CANCEL, self.id)
        print('trader sent:', send)
        return send

    def balance_and_position(self):
        send = (ActionType.BALANCE, self.id)
        print('trader sent:', send)
        return send

    def process_response(self, response):
        if response[0] == 2:
            print('Trader {} wants to amend the order'.format(response[1]))
            return self.amend_quantity(response[2])
        
        elif response[0] == 3:
            print('Trader {} wants to cancel the order'.format(response[1]))
            return self.cancel_order()
        
        elif response[0] == 4:
            print('Trader {} wants to check the balance'.format(response[1]))
            return self.balance_and_position()
        
        elif response[0] == 1:
            if response[4] == OrderType.LIMIT: 
                print('Trader {} wants to place limit order'.format(response[1]))
                return self.place_limit_order(response[2], response[3], response[5])
            
            elif response[4] == OrderType.MARKET: 
                print('Trader {} wants to place market order'.format(response[1]))
                return self.place_market_order(response[2], response[5])
            
            elif response[4] == OrderType.IOC: 
                print('Trader {} wants to place ioc order'.format(response[1]))
                return self.place_ioc_order(response[2], response[3], response[5])
        else:
            raise UndefinedResponse("Undefined Response Received!")

    def random_action(self):
        import random
        
        if self.book_position == 0:
            
            action_num = random.randint(1,4)
            if action_num == 2:
                action_num = 1
            elif action_num == 3:
                action_num = 4
            
            side_num = random.randint(1,2)
            if side_num == 1:
                side = OrderSide.BUY
            elif side_num == 2:
                side = OrderSide.SELL
                
            order_type_num = random.randint(1,3)
            if order_type_num == 1:
                order_type = OrderType.LIMIT
            elif order_type_num == 2:
                order_type = OrderType.MARKET
            elif order_type_num == 3:
                order_type = OrderType.IOC
                
            quantity = random.randint(1,20)
            price = random.randint(60,70)
            
            request = (action_num, self.id, quantity, price, order_type, side)
            request = (3, self.id, quantity, price, OrderType.LIMIT, side)
            # request - (Action #, Trader ID, Additional Arguments)
            
            return self.process_response(request)
            
        elif self.book_position == 1:
            
            action_num = random.randint(2,3)
                
            quantity = random.randint(1,20)
            price = random.randint(60,70)
            
            request = (action_num, self.id, quantity)
            
            return self.process_response(request)
        

    def run_infinite_loop(self):
        request = self.random_action()
        self.exchange.handle_request(request)
#         while self.balance_track[0] > 0:
#         for i in range(3):
#         request = self.random_action()
#             Exchange.handle_request(self, self.random_action())
#         pass
        # The trader needs to continue to take actions until the book balance falls to 0
        # While the trader can take actions, it chooses from a random_action and uploads the action
        # to the exchange
        
        #The trader then takes any received responses from the exchange and processes it

In [259]:
class Exchange(MyThread):
    def __init__(self):
        super().__init__()
        self.balance = [1000000 for _ in range(100)]
        self.position = [0 for _ in range(100)]
        self.matching_engine = MatchingEngine()

    def place_new_order(self, order):
#         self.matching_engine.handle_order(order[1])
        if order.type.value == 1:
            filled_orders = self.matching_engine.handle_limit_order(order)
        elif order.type.value == 2:
            filled_orders = self.matching_engine.handle_market_order(order)
        elif order.type.value == 3:
            filled_orders = self.matching_engine.handle_ioc_order(order)
          
        if len(filled_orders) != 0:
            self.balance[order.id] -= filled_orders[0].quantity * filled_orders[0].price
            self.position[order.id] += filled_orders[0].quantity 

        processed = (order.id, (ActionType.PLACE, order))
        # (Trader id that processed the order, (action type enum, order))
        results = []
        results.append(processed)
        return results

    def amend_quantity(self, id, quantity):
        logical = False
        try:
            self.matching_engine.amend_quantity(id, quantity)
            logical = True
            processed = (ActionType.AMEND, logical)
            return processed
        except:
            processed = (ActionType.AMEND, logical)
            return processed

    def cancel_order(self, id):
        logical = False
        try:
            self.matching_engine.cancel_order(id)
            logical = True
            processed = (ActionType.CANCEL, logical)
            return processed
        except:
            processed = (ActionType.CANCEL, logical)
            return processed

    def balance_and_position(self, id):
        # The matching engine must be able to process the 'balance' action based on the given parameters
        
        # The return must be in the form (action type enum, (trader balance, trader positions))
        pass

    def handle_request(self, request):
        print('exch receiv:', request)
        if request[0].value == 1:
            print('Trader {} wants to place the order'.format(request[1]))
            self.place_new_order(request[2])
        elif request[0].value == 2: # amend
            print('Trader {} wants to amend the order'.format(request[1]))
            self.amend_quantity(request[1], request[2])
        elif request[0].value == 3: # cancel
            print('Trader {} wants to cancel the order'.format(request[1]))
            self.cancel_order(request[1])
        elif request[0].value == 4: # balance
            print('Trader {} wants to check the balance'.format(request[1]))
            self.balance_and_position(request[1])
        else:
            raise UndefinedTraderAction("Undefined Trader Action!")

    def run_infinite_loop(self):
        # The exchange must continue handling orders as orders are issued by the traders
        # A way to do this is check if there are any orders waiting to be processed in the deque
        
        # If there are, handle the request using the functions built above and using the
        # corresponding trader's deque, return an acknowledgement based on the response
        pass
    
marina = Trader(1)
print(marina.book_position)
print(marina.balance_track)
marina.run_infinite_loop()

0
[1000000]
Trader 1 wants to cancel the order
trader sent: (<ActionType.CANCEL: 3>, 1)
exch receiv: (<ActionType.CANCEL: 3>, 1)
Trader 1 wants to cancel the order
goo
(<ActionType.CANCEL: 3>, True)


In [ ]:
if __name__ == "__main__":

    trader = [Trader(i) for i in range(100)]
    exchange = Exchange()

    exchange.start()
    for t in trader:
        t.start()

    exchange.join()
    for t in trader:
        t.join()

    sum_exch = 0
    for t in MyThread.list_of_threads:
        if t.id == "NoID":
            for b in t.balance:
                sum_exch += b

    print("Total Money Amount for All Traders before Trading Session: " + str(sum_exch))

    for i in range(10000):
        thread_active = False
        for t in MyThread.list_of_threads:
            if t.is_started:
                t.run_infinite_loop()
                thread_active = True
        if not thread_active:
            break

    sum_exch = 0
    for t in MyThread.list_of_threads:
        if t.id == "NoID":
            for b in t.balance:
                sum_exch += b

    print("Total Money Amount for All Traders after Trading Session: ", str(int(sum_exch)))